# Basic Statistics

In [8]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [2]:
!pip install pyspark==3.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 9.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 14.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764010 sha256=7fe3911dd0f64dcece12bb168c10cd66c4882f1eda1372d1477f77490b66ac32
  Stored in directory: /Users/Elena_Sidorova/Library/Caches/pip/wheels/05/75/73/81f84d174299abca38dd6a06a5b98b08ae25fce50ab8986fa1
Successfully built pyspark


In [1]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [2]:
sc = SparkContext.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 10:35:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark = SparkSession \
    .builder \
    .getOrCreate()

Let's start with the first function and calculate the minimal temperature for the test data set we have created. 

In [4]:
def minTemperature():
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return spark.sql("SELECT min(temperature) as mintemp from washing").first().mintemp

Now let's do the same for the mean of the temperature

In [7]:
def meanTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return spark.sql("SELECT avg(temperature) as meantemp from washing").first().meantemp

Let's do the same for the maximum of the temperature

In [8]:
def maxTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    return spark.sql("SELECT max(temperature) as maxtemp from washing").first().maxtemp

Let's do the same for the standard deviation of the temperature

In [9]:
def sdTemperature():
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    #https://spark.apache.org/docs/2.3.0/api/sql/
    return spark.sql("SELECT std(temperature) as sdtemp from washing").first().sdtemp

Let's do the same for the skew of the temperature. Also note that we are making use of python's string formatting capabilitis where the results of the two function calls to "meanTemperature" and "sdTemperature" are inserted at the "%s" symbols in the SQL string.

In [25]:
spark.sql("SELECT count(temperature) as count from washing").show()

+-----+
|count|
+-----+
| 1342|
+-----+



In [43]:
def skewTemperature():    
    return spark.sql(f"""
SELECT 
    (
        1/count(temperature)
    ) *
    SUM (
        POWER(temperature - %s,3)/POWER(%s,3)
    )
as sktemperature from washing
                    """ %(meanTemperature(),sdTemperature())).first().sktemperature

In [41]:
def skewTemperature():    
    return spark.sql("""
SELECT 
    skewness(temperature)
as sktemperature from washing
                    """ %(meanTemperature(),sdTemperature())).first().sktemperature

Kurtosis is the 4th statistical moment.

In [45]:
def kurtosisTemperature():    
        return spark.sql("""
SELECT 
    (
        1/count(temperature)
    ) *
    SUM (
        POWER(temperature - %s,4)/POWER(%s,4)
    )
as ktemperature from washing
                    """ %(meanTemperature(),sdTemperature())).first().ktemperature


In [12]:
def correlationTemperatureHardness():
    #some reference: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame
    #https://spark.apache.org/docs/2.3.0/api/sql/
    return spark.sql("SELECT corr(temperature,hardness) as temperaturehardness from washing").first().temperaturehardness

Now it is time to grab a PARQUET file and create a dataframe out of it. Using SparkSQL we can handle it like a database. 

In [13]:
!wget https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
!mv washing.parquet?raw=true washing.parquet

zsh:1: no matches found: https://github.com/IBM/coursera/blob/master/coursera_ds/washing.parquet?raw=true
zsh:1: no matches found: washing.parquet?raw=true


In [17]:
df = spark.read.parquet('washing.parquet')
df.createOrReplaceTempView('washing')
df.show()

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0d86485d0f88d1f9d...|1-57940679fb8a713...|    4|      11|acceptable|     null|      77| null|        100|1547808723923|   null|
|0d86485d0f88d1f9d...|1-15ff3a0b304d789...|    2|    null|      null|     null|    null| 1046|       null|1547808729917|   null|
|0d86485d0f88d1f9d...|1-97c2742b68c7b07...|    4|    null|      null|       71|    null| null|       null|1547808731918|    236|
|0d86485d0f88d1f9d...|1-eefb903dbe45746...|   19|      11|acceptable|     null|      75| null|         86|1547808738999|   null|
|0d86485d0f88d1f9d...|1-5f68b4c72813c25...|    7|    null|      null|       75|    null| null|   

Now let's test the functions you've implemented

In [18]:
min_temperature = 0
mean_temperature = 0
max_temperature = 0
sd_temperature = 0
skew_temperature = 0
kurtosis_temperature = 0
correlation_temperature = 0


In [19]:
min_temperature = minTemperature()
print(min_temperature)

80


In [20]:
mean_temperature = meanTemperature()
print(mean_temperature)

90.03800298062593


In [21]:
max_temperature = maxTemperature()
print(max_temperature)

100


In [22]:
sd_temperature = sdTemperature()
print(sd_temperature)

6.1007610586219725


In [44]:
skew_temperature = skewTemperature()
print(skew_temperature)

0.010398374583336334


In [46]:
kurtosis_temperature = kurtosisTemperature()
print(kurtosis_temperature)

1.773427150876985


In [47]:
correlation_temperature = correlationTemperatureHardness()
print(correlation_temperature)

0.017754069047296324


Congratulations, you are done, please submit this notebook to the grader. 
We have to install a little library in order to submit to coursera first.

Then, please provide your email address and obtain a submission token on the grader’s submission page in coursera, then execute the subsequent cells

### Note: We've changed the grader in this assignment and will do so for the others soon since it gives less errors
This means you can directly submit your solutions from this notebook

In [48]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2023-03-09 11:01:01--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: 'rklib.py'

rklib.py            100%[===================>]   2.48K  --.-KB/s    in 0.001s  

2023-03-09 11:01:02 (2.64 MB/s) - 'rklib.py' saved [2540/2540]



In [49]:
from rklib import submitAll
import json

key = "Suy4biHNEeimFQ479R3GjA"
email = "elleneug@gmail.com"
token = "u2GzadwOQBIlvZ0O"



In [50]:
parts_data = {}
parts_data["FWMEL"] = json.dumps(min_temperature)
parts_data["3n3TK"] = json.dumps(mean_temperature)
parts_data["KD3By"] = json.dumps(max_temperature)
parts_data["06Zie"] = json.dumps(sd_temperature)
parts_data["Qc8bI"] = json.dumps(skew_temperature)
parts_data["LoqQi"] = json.dumps(kurtosis_temperature)
parts_data["ehNGV"] = json.dumps(correlation_temperature)



submitAll(email, token, key, parts_data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"TzU1P","id":"sUpST4RAEeawAApvKZgcCQ~TzU1P~dE05T743Ee2nRBJiYOmnzQ","courseId":"sUpST4RAEeawAApvKZgcCQ"}],"paging":{},"linked":{}}
-------------------------
